# Takes data scraped from Swiss Jazz Radio and saves it to a playlist on Spotify

First, read the CSV file:

In [1]:
# import numpy as np
import pandas as pd

df = pd.read_csv(os.path.join('scrapes', 'sjr_tracks.csv'), names=['Title', 'Artist'])
df['Spotify ID'] = None
df = df[0:100]   # better limit this as file is big!
df.head()

,Title,Artist,Spotify ID
0,A Ballad,Gerry Mulligan & Stan Getz,None
1,A Beautiful Friendship,Jeremy Davenport,None
2,A Beautiful Friendship,Johnny Varro Trio,None
3,A Bench In The Park,Ralf Ruh Trio,None
4,A Brush With Bunj,James Morrison,None


Login to Spotify:

In [2]:
import os
import spotify

sp = spotify.login(scope='playlist-read-private playlist-modify-private')
user = sp.current_user()
user['display_name']   # check if login successful

'Mehmet Birgi'

Get the spotify ids for the tracks:

In [3]:
# import json
results = []
for index, row in df.iterrows():
    name, title = row['Title'], row['Artist']
    q = f"{name} {title}"
    row_results = sp.search(q, type='track', market='CH')
    items = row_results['tracks']['items']
    if items:
        df['Spotify ID'][index] = items[0]['id']
df.head()

,Title,Artist,Spotify ID
0,A Ballad,Gerry Mulligan & Stan Getz,7yLckxzDe6ktaWz9UbpXcK
1,A Beautiful Friendship,Jeremy Davenport,5a9VN1uwPalg7dNMFlAjOO
2,A Beautiful Friendship,Johnny Varro Trio,7cxbHzIW3UD3I9qhA7HFD1
3,A Bench In The Park,Ralf Ruh Trio,0R9yNJMys9Qiagz885MCBg
4,A Brush With Bunj,James Morrison,75slKFembjAgcPipi6qbJ0


Get the playlist to edit:

In [4]:
playlist_id, is_new = spotify.get_playlist_by_name(sp, "Swiss Jazz Radio Tracks", create_if_none=True)
if playlist_id and is_new:
    print(f"Created new playlist with ID {playlist_id}")
elif playlist_id:
    print(f"Found existing playlist with ID {playlist_id}")
else:
    print("Something went wrong :-(")

Found existing playlist with ID 77523QiGKT14yvmZM4bJl9


Append tracks (skipping duplicates):

In [6]:
tracks_to_add = [row['Spotify ID'] for index, row in df.iterrows() if row['Spotify ID'] is not None]
num_tracks_added = spotify.add_tracks(sp, playlist_id, tracks_to_add)
print(f"Number of tracks added to playlist: {num_tracks_added}")

Number of tracks added to playlist: 0
